# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType

import os
import time
import uuid
from datetime import datetime
from datetime import date
import requests

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util

[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ./data/2021-10-29-12:02:42  Created 
Directory  ./data/2021-10-29-12:02:42/image  Created 


In [3]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-10-29 12:02:44,435 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-10-29 12:02:44,436 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain
2021-10-29 12:02:45,940 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain
2021-10-29 12:02:45,944 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-10-29 12:02:46,674 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-10-29 12:02:49,345 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-10-29 12:02:49,347 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-10-29 12:02:49,349 -    DEBUG -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-10-29 12:02:49,352 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-10-29 12:02:49,591 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query
2021-10-29 12:02:49,636 -     INF

In [4]:
def process_text_and_think (scenario: Scenario, 
                  place_id:str, 
                  location: str, 
                  textSignal: TextSignal,
                  human_id: str,
                  my_brain:LongTermMemory):
    thoughts = ""
    chat = Chat(human_id)
    chat.add_utterance([UtteranceHypothesis(c_util.seq_to_text(textSignal.seq), 1.0)])
    chat.last_utterance.analyze()
    # No triple was extracted, so we missed three items (s, p, o)
    if chat.last_utterance.triple is None:
        utterance = "Any gossip?" + '\n'
    else:
        triple = c_util.rephrase_triple_json_for_capsule(chat.last_utterance.triple)
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_utterance_and_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal, 
                                                                  human_id,
                                                                  chat.last_utterance.perspective, 
                                                                  triple)
        print('Capsule:', capsule)
        thoughts = my_brain.update(capsule, reason_types=True, create_label=False)
        #print(thoughts)
    return thoughts  

In [5]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + HUMAN_NAME + "?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)
    thoughts = process_text_and_think (scenario, 
                  place_id, 
                  location, 
                  textSignal, 
                  HUMAN_ID, my_brain)
    
    # Create the response from the system and store this as a new signal
    utterance = "" #ttt.getTextFromTriples(response)
    if not utterance:
            utterance = f"So you what do you want to talk about {HUMAN_NAME}?\n"

    response = utterance[::-1]
    print(AGENT + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')


Leolani2: Hi there. Who are you Stranger?



 I am Piek


Stranger: I am Piek
2021-10-29 12:03:06,810 -     INFO - cltl.triple_extraction.api.Chat (VOID)              000 - << Start of Chat with VOID >>
2021-10-29 12:03:06,812 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-10-29 12:03:06,813 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-10-29 12:03:06,822 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-10-29 12:03:07,937 -     INFO - cltl.triple_extraction.api.Chat (VOID)              001 -       VOID: "I am Piek"
{'subject': {'label': 'VOID', 'type': ['noun.state']}, 'predicate': {'label': 'be', 'type': ['verb.stative']}, 'object': {'label': 'Piek', 'type': ['person']}}
{'subject': {'state', 'VOID'}, 'predicate': {'be', 'stative'}, 'object': {'Piek', 'person'}}
Capsule: {'chat': '2021-10-29-12:02:42', 'turn': '754919a2-3d25-4204-a423-e3cf1978c443', 'author': 'VOID', 'utterance': 'I am Piek', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position


 I like pizza


2021-10-29 12:03:20,825 -     INFO - cltl.triple_extraction.api.Chat (VOID)              000 - << Start of Chat with VOID >>
2021-10-29 12:03:21,768 -     INFO - cltl.triple_extraction.api.Chat (VOID)              001 -       VOID: "I like pizza"
{'subject': {'label': 'VOID', 'type': ['noun.state']}, 'predicate': {'label': 'like', 'type': ['verb.emotion']}, 'object': {'label': 'pizza', 'type': ['noun.food']}}
{'subject': {'state', 'VOID'}, 'predicate': {'like', 'emotion'}, 'object': {'pizza', 'food'}}
Capsule: {'chat': '2021-10-29-12:02:42', 'turn': '471d13b8-4448-4d94-986a-811796f2f225', 'author': 'VOID', 'utterance': 'I like pizza', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '0-12', 'subject': {'label': 'piek', 'type': 'person'}, 'predicate': {'type': 'see'}, 'object': {'label': 'pills', 'type': 'object'}, 'perspective': {'sentiment': '0.75', 'certainty': 1, 'polarity': 1, 'emotion': <Emotion.NEUTRAL: 7>}, 'context_id': 'Leolani2', 'date': datetime.date(2021, 10, 29)


 bye


## End of notebook